# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [ ]:
!pip -q install tqdm boto3 requests regex sentencepiece sacremoses datasets

그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [5]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('openai-gpt', clean_up_tokenization_spaces=True)
tokenizer.pad_token = tokenizer.unk_token

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [18]:
ds = load_dataset("fancyzhx/ag_news")
# ds = load_dataset("stanfordnlp/imdb")
from torch.utils.data import Subset

def collate_fn(batch):
  texts = [row['text'] for row in batch]
  labels = [row['label'] for row in batch]

  encoded = tokenizer(texts, padding=True, truncation=False, return_tensors='pt')
  texts = encoded['input_ids']
  masks = encoded['attention_mask']
  labels = torch.LongTensor(labels)

  return texts, masks, labels


ds = load_dataset("fancyzhx/ag_news")
# BATCH_SIZE = len(ds['train'])
BATCH_SIZE = 32
train_loader = DataLoader(
    ds['train'], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)
# train_loader = DataLoader(
#     Subset(ds['train'],range(0,32)), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
# )
# test_loader = DataLoader(
#     Subset(ds['test'], range(0,32)), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
# )
# for inputs, masks, labels in train_loader:
#     print(inputs.shape)

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [11]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'openai-gpt')
model.config

Using cache found in /Users/heekyung/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

OpenAIGPTConfig {
  "_name_or_path": "openai-gpt",
  "afn": "gelu",
  "architectures": [
    "OpenAIGPTLMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "openai-gpt",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 512,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.44.1",
  "vocab_size": 40478
}

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [15]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'openai-gpt')

    self.classifier = nn.Linear(self.encoder.config.n_embd, 4)
    self.dropout = nn.Dropout(self.encoder.config.embd_pdrop)
    for param in self.encoder.parameters():
        param.requires_grad = False
      
  def forward(self, input_ids, attention_mask=None):
    x = self.encoder(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
    logits = self.classifier(x[:,-1])
    # cls_embedding = outputs.last_hidden_state[:, 0, :]  # Get the [CLS] token's output (batch_size, hidden_size)
    # x = self.dropout(outputs)
    return logits
        
model = TextClassifier()

Using cache found in /Users/heekyung/.cache/torch/hub/huggingface_pytorch-transformers_main


위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [ ]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [19]:
import itertools
from datetime import datetime
from torch.optim import Adam

lr = 1e-3
loss_fn = nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'mps'
model = model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 5

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for (inputs, masks, labels) in itertools.islice(train_loader, 100):
    model.zero_grad()
    inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device).float()
    preds = model(inputs, masks)
    # preds = model(inputs, masks)[..., 0] # loss over 70,000
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 78.746238976717
Epoch   1 | Train Loss: 53.24991285800934
Epoch   2 | Train Loss: 49.69931522011757
Epoch   3 | Train Loss: 46.91623172163963
Epoch   4 | Train Loss: 43.74498933553696


In [20]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for (inputs, masks, labels) in dataloader:
    inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
    preds = model(inputs, 
                  
                  
                  masks)
    preds = torch.argmax(preds, dim=-1)
    # preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.852 | Test acc: 0.846


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.

In [ ]:
# ds = load_dataset("fancyzhx/ag_news")

# def collate_fn(batch):
#   texts = [rows['text'] for rows in batch]
#   texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=False ).input_ids)
#   return texts, labels

# train_loader = DataLoader(
#     ds['train'], batch_size=len(ds['train']), shuffle=True, collate_fn=collate_fn
# )

# for idx, (inputs, labels) in enumerate(train_loader):
#     print(idx, inputs.shape)